In [2]:
## 1. c. LOOCV

In [3]:
## 2. d. high bias and high variance

In [4]:
## 3. a. low bias and low variance

In [5]:
## 4. Regularization is to restrict the estimation of coefficients by putting penalty on them
## so the fitted model has lower variability or is simpler. Lower variabiliby helps produce
## more accurate result, while simpler models are easier to interpret with less variables.

In [50]:
## 5
## import packages and modules
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

## fetch file content from s3
s3 = boto3.resource('s3')
bucket = s3.Bucket('danhtran358-data-445-bucket')
bucket_object = bucket.Object('framingham.csv')
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## read file content to data-frame
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [51]:
## remove observations with missing values
heart = heart.dropna().reset_index(drop = True)

In [52]:
## defining input and target variables
X1 = heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
X2 = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

In [53]:
## scale input variables to 0-1 range
X1_scaled = MinMaxScaler().fit_transform(X1)
X2_scaled = MinMaxScaler().fit_transform(X2)

## recreate data frame from arrays
X1_scaled = pd.DataFrame(X1_scaled)
X2_scaled = pd.DataFrame(X2_scaled)

In [54]:
## lists to store results
md1_result = list()
md2_result = list()

kfold = KFold(n_splits = 5, shuffle = True)

for train_ix, test_ix in kfold.split(X):
    ## split data into train and validation folds
    X1_train, X1_test = X1_scaled.loc[train_ix], X1_scaled.loc[test_ix]
    X2_train, X2_test = X2_scaled.loc[train_ix], X2_scaled.loc[test_ix]
    Y_train, Y_test = Y.loc[train_ix], Y.loc[test_ix]
    
    ## build logistic models
    logit_md1 = LogisticRegression().fit(X1_train, Y_train)
    logit_md2 = LogisticRegression().fit(X2_train, Y_train)
    
    ## predict the likelihood
    preds1 = logit_md1.predict_proba(X1_test)[:,1]
    preds2 = logit_md2.predict_proba(X2_test)[:,1]
    
    ## label likelihood
    preds1 = np.where(preds1 < 0.25, 0, 1)
    preds2 = np.where(preds2 < 0.25, 0, 1)
    
    ## compute and store F1 score
    md1_result.append(f1_score(Y_test, preds1))
    md2_result.append(f1_score(Y_test, preds2))

print(md1_result)
print(md2_result)  

[0.32456140350877194, 0.34821428571428575, 0.2938388625592417, 0.3486238532110092, 0.3463203463203463]
[0.3225806451612903, 0.28444444444444444, 0.3380281690140845, 0.30434782608695654, 0.32799999999999996]


In [55]:
## compute average F1 scores
avg_f1_score1 = np.mean(md1_result)
avg_f1_score2 = np.mean(md2_result)

print(avg_f1_score1)
print(avg_f1_score2)

## I would use the first model to predict TenYearCHD because it has higher F1 score,
## which means it would predict with higher accuracy

0.33231175026273096
0.3154802169413552
